# Assignment 1

You only need to write one line of code for each question. When answering questions that ask you to identify or interpret something, the length of your response doesn’t matter. For example, if the answer is just ‘yes,’ ‘no,’ or a number, you can just give that answer without adding anything else.

We will go through comparable code and concepts in the live learning session. If you run into trouble, start by using the help `help()` function in Python, to get information about the datasets and function in question. The internet is also a great resource when coding (though note that **no outside searches are required by the assignment!**). If you do incorporate code from the internet, please cite the source within your code (providing a URL is sufficient).

Please bring questions that you cannot work out on your own to office hours, work periods or share with your peers on Slack. We will work with you through the issue.

### Classification using KNN

Let's set up our workspace and use the **Wine dataset** from `scikit-learn`. This dataset contains 178 wine samples with 13 chemical features, used to classify wines into different classes based on their origin.

The **response variable** is `class`, which indicates the type of wine. We'll use all of the chemical features to predict this response variable.

In [59]:
# Import standard libraries
import pandas as pd
import numpy as np
import random
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import recall_score, precision_score
from sklearn.model_selection import cross_validate
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score

In [60]:
from sklearn.datasets import load_wine

# Load the Wine dataset
wine_data = load_wine()

# Convert to DataFrame
wine_df = pd.DataFrame(wine_data.data, columns=wine_data.feature_names)

# Bind the 'class' (wine target) to the DataFrame
wine_df['class'] = wine_data.target

# Display the DataFrame
wine_df


,alcohol,malic_acid,ash,alcalinity_of_ash,magnesium,total_phenols,flavanoids,nonflavanoid_phenols,proanthocyanins,color_intensity,hue,od280/od315_of_diluted_wines,proline,class
0,14.23,1.71,2.43,15.6,127.0,2.80,3.06,0.28,2.29,5.64,1.04,3.92,1065.0,0
1,13.20,1.78,2.14,11.2,100.0,2.65,2.76,0.26,1.28,4.38,1.05,3.40,1050.0,0
2,13.16,2.36,2.67,18.6,101.0,2.80,3.24,0.30,2.81,5.68,1.03,3.17,1185.0,0
3,14.37,1.95,2.50,16.8,113.0,3.85,3.49,0.24,2.18,7.80,0.86,3.45,1480.0,0
4,13.24,2.59,2.87,21.0,118.0,2.80,2.69,0.39,1.82,4.32,1.04,2.93,735.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
173,13.71,5.65,2.45,20.5,95.0,1.68,0.61,0.52,1.06,7.70,0.64,1.74,740.0,2
174,13.40,3.91,2.48,23.0,102.0,1.80,0.75,0.43,1.41,7.30,0.70,1.56,750.0,2
175,13.27,4.28,2.26,20.0,120.0,1.59,0.69,0.43,1.35,10.20,0.59,1.56,835.0,2
176,13.17,2.59,2.37,20.0,120.0,1.65,0.68,0.53,1.46,9.30,0.60,1.62,840.0,2


#### **Question 1:** 
#### Data inspection

Before fitting any model, it is essential to understand our data. **Use Python code** to answer the following questions about the **Wine dataset**:

_(i)_ How many observations (rows) does the dataset contain?

In [61]:
# Your answer here
# Answer: 178 rows
len(wine_df)
# I also grabbed this info from the output for wine_df in the above code snippet and I ran wine_df.info() 

178

_(ii)_ How many variables (columns) does the dataset contain?

In [62]:
# Your answer here
# Answer: 14 columns
len(wine_df.columns)
# I also grabbed this info from the output for wine_df in the above code snippet and I ran wine_df.info() 

14

_(iii)_ What is the 'variable type' of the response variable `class` (e.g., 'integer', 'category', etc.)? What are the 'levels' (unique values) of the variable?

In [63]:
# Your answer here
# Answer:  variable type = integer (int64) and the levels (unique values) of class are [0, 1, 2]
# I used wine_df['class'].value_counts()  to grab this info
print("Levels (unique values) of the class:")
wine_df['class'].value_counts()

Levels (unique values) of the class:


class
1    71
0    59
2    48
Name: count, dtype: int64


_(iv)_ How many predictor variables do we have (Hint: all variables other than `class`)? 

In [64]:
# Your answer here
# Answer: 13 predictor variables
len(wine_df.columns) - 1

13

You can use `print()` and `describe()` to help answer these questions.

#### **Question 2:** 
#### Standardization and data-splitting

Next, we must preform 'pre-processing' or 'data munging', to prepare our data for classification/prediction. For KNN, there are three essential steps. A first essential step is to 'standardize' the predictor variables. We can achieve this using the scaler method, provided as follows:

In [65]:
# Select predictors (excluding the last column)
predictors = wine_df.iloc[:, :-1]

# Standardize the predictors
scaler = StandardScaler()
predictors_standardized = pd.DataFrame(scaler.fit_transform(predictors), columns=predictors.columns)

# Display the head of the standardized predictors
print(predictors_standardized.head())

    alcohol  malic_acid       ash  alcalinity_of_ash  magnesium  \
0  1.518613   -0.562250  0.232053          -1.169593   1.913905   
1  0.246290   -0.499413 -0.827996          -2.490847   0.018145   
2  0.196879    0.021231  1.109334          -0.268738   0.088358   
3  1.691550   -0.346811  0.487926          -0.809251   0.930918   
4  0.295700    0.227694  1.840403           0.451946   1.281985   

   total_phenols  flavanoids  nonflavanoid_phenols  proanthocyanins  \
0       0.808997    1.034819             -0.659563         1.224884   
1       0.568648    0.733629             -0.820719        -0.544721   
2       0.808997    1.215533             -0.498407         2.135968   
3       2.491446    1.466525             -0.981875         1.032155   
4       0.808997    0.663351              0.226796         0.401404   

   color_intensity       hue  od280/od315_of_diluted_wines   proline  
0         0.251717  0.362177                      1.847920  1.013009  
1        -0.293321  0.406051

(i) Why is it important to standardize the predictor variables?

> Your answer here...
Some predictor variables have values with wider range of values. Consequently, this could result in some variables having potentially more dominance on the distance calculation using the euclidean formula, as was seen in our in-class lab session with perimeter mean having a stronger influence than concavity mean. In this dataset, some columns have numbers that are of a higher magnitude than some, for example, proline has larger numbers than hue. Standardizing the predictor variables helps to prevent bias by giving all the predictor variables a more equal playing field.

In [66]:
print("Hue has values that are smaller than proline")
print("Hue")
print(wine_df['hue'].unique())
print("Proline")
print(wine_df['proline'].unique())

Hue has values that are smaller than proline
Hue
[1.04  1.05  1.03  0.86  1.02  1.06  1.08  1.01  1.25  1.17  1.15  1.2
 1.28  1.07  1.13  1.23  0.96  1.09  1.11  1.12  0.92  1.19  1.1   1.18
 0.89  0.95  0.91  0.88  0.82  0.87  1.24  0.98  0.94  1.22  1.45  0.906
 1.36  1.31  0.99  1.38  1.16  0.84  0.79  1.33  1.    1.42  1.27  0.8
 0.75  0.9   0.93  1.71  0.7   0.73  0.69  0.97  0.76  0.74  0.66  0.78
 0.81  0.77  0.65  0.6   0.58  0.54  0.55  0.57  0.59  0.48  0.61  0.56
 0.67  0.68  0.85  0.72  0.62  0.64 ]
Proline
[1065. 1050. 1185. 1480.  735. 1450. 1290. 1295. 1045. 1510. 1280. 1320.
 1150. 1547. 1310. 1130. 1680.  845.  780.  770. 1035. 1015.  830. 1195.
 1285.  915. 1515.  990. 1235. 1095.  920.  880. 1105. 1020.  760.  795.
  680.  885. 1080.  985. 1060. 1260. 1265. 1190. 1375. 1120.  970. 1270.
  520.  450.  630.  420.  355.  678.  502.  510.  750.  718.  870.  410.
  472.  886.  428.  392.  500.  463.  278.  714.  515.  495.  562.  625.
  480.  290.  345.  937.  660.  406.

(ii) Why did we elect not to standard our response variable `Class`?

> Your answer here...
The unique values for class are 0, 1, 2, and they seem more like categorical data, similar to 'Benign' and 'Malignant' values from our in-class lab sessions. So they are not the same sort of numerical continuous data like the other columns.

In [67]:
print(wine_df['class'].unique())

[0 1 2]


(iii) A second essential step is to set a random seed. Do so below (Hint: use the random.seed function). Why is setting a seed important? Is the particular seed value important? Why or why not?

> Your answer here...
When we use random.seed, the result stay the same each time we run our code with the same seed value even though we are using random, thus helping us with reproducibility. This is useful for testing and comparison. At the start, we could choose any integer as the seed value, and every time we re-run the code, the result will be the same. We should only change the seed if we want a different random set, and after changing the seed, we could repeat the previous experiment by changing the seed back to the previous value. If we only need one seed value, the number that we choose doesn't matter, but if we are running multiple rounds of random experiments, remembering the seed number to return to a previous state is important. 

(iv) A third essential step is to split our standardized data into separate training and testing sets. We will split into 75% training and 25% testing. The provided code randomly partitions our data, and creates linked training sets for the predictors and response variables. 

Extend the code to create a non-overlapping test set for the predictors and response variables.

In [68]:
# set a seed for reproducibility
np.random.seed(3)

# split the data into a training and testing set. hint: use train_test_split !

# Your code here ...
# The last column (class) was removed from the 'predictors' variable
# Hence, first we will create a a dataset with the standardized predictor variable and the response variable
standardized_wine = wine_df.copy()
column_to_exclude = ['class']
columns_to_scale = standardized_wine.columns.difference(column_to_exclude)
scaler = StandardScaler()
standardized_wine[columns_to_scale] = scaler.fit_transform(wine_df[columns_to_scale])
print(standardized_wine.info())
# Using wine_df["class"].value_counts(), it could be seen that class has 71% of the value as 1, 59% as 0 and 48% as 2. So, we will try to keep that same ratio with stratify.
# we will also shuffle the values for extra measures
# splitting data into 75% training and 25% testing sets
wine_train, wine_test = train_test_split(standardized_wine, train_size = 0.75, shuffle= True, stratify = standardized_wine["class"])
print("Training Set")
print(wine_train.shape)
print("Testing Set")
print(wine_test.shape)


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 178 entries, 0 to 177
Data columns (total 14 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   alcohol                       178 non-null    float64
 1   malic_acid                    178 non-null    float64
 2   ash                           178 non-null    float64
 3   alcalinity_of_ash             178 non-null    float64
 4   magnesium                     178 non-null    float64
 5   total_phenols                 178 non-null    float64
 6   flavanoids                    178 non-null    float64
 7   nonflavanoid_phenols          178 non-null    float64
 8   proanthocyanins               178 non-null    float64
 9   color_intensity               178 non-null    float64
 10  hue                           178 non-null    float64
 11  od280/od315_of_diluted_wines  178 non-null    float64
 12  proline                       178 non-null    float64
 13  class

#### **Question 3:**
#### Model initialization and cross-validation
We are finally set to fit the KNN model. 


Perform a grid search to tune the `n_neighbors` hyperparameter using 10-fold cross-validation. Follow these steps:

1. Initialize the KNN classifier using `KNeighborsClassifier()`.
2. Define a parameter grid for `n_neighbors` ranging from 1 to 50.
3. Implement a grid search using `GridSearchCV` with 10-fold cross-validation to find the optimal number of neighbors.
4. After fitting the model on the training data, identify and return the best value for `n_neighbors` based on the grid search results.

In [71]:
# Your code here...
# just adding n_neighbors= 3 for now
knn = KNeighborsClassifier(n_neighbors= 3)
parameter_grid = {
    "n_neighbors": range(1, 51), # n_neighbors ranging from 1 to 50
}

wine_tune_grid = GridSearchCV(
    estimator=knn,
    param_grid=parameter_grid,
    cv=10
)

# Fitting the model pn the training data
wine_tune_grid.fit(wine_train.iloc[:, :-1], wine_train["class"])

# The best value for k
best_value_for_k = wine_tune_grid.best_params_['n_neighbors']
print(f"Best Value for K: {best_value_for_k }")

# This wasn't asked but just performing the data visualization for more insight
accuracy_grid = pd.DataFrame(wine_tune_grid.cv_results_)
accuracy_grid.info()

# will only display the graph when testing in Visual Studio code. It might be causing the flickering in the browser (Github)
# Create the plot
#plt.figure(figsize=(10, 6))

# Plot mean test scores with error bars
#plt.plot(accuracy_grid['param_n_neighbors'], accuracy_grid['mean_test_score'], '-o', color='blue')

# Add labels and legend
#plt.xlabel('Number of Neighbors')
#plt.ylabel('Accuracy estimate')
#plt.title('K-Nearest Neighbors Performance')
#plt.tight_layout()
#plt.show()




Best Value for K: 25
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50 entries, 0 to 49
Data columns (total 19 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   mean_fit_time      50 non-null     float64
 1   std_fit_time       50 non-null     float64
 2   mean_score_time    50 non-null     float64
 3   std_score_time     50 non-null     float64
 4   param_n_neighbors  50 non-null     int64  
 5   params             50 non-null     object 
 6   split0_test_score  50 non-null     float64
 7   split1_test_score  50 non-null     float64
 8   split2_test_score  50 non-null     float64
 9   split3_test_score  50 non-null     float64
 10  split4_test_score  50 non-null     float64
 11  split5_test_score  50 non-null     float64
 12  split6_test_score  50 non-null     float64
 13  split7_test_score  50 non-null     float64
 14  split8_test_score  50 non-null     float64
 15  split9_test_score  50 non-null     float64
 16  mean_te

#### **Question 4:**
#### Model evaluation

Using the best value for `n_neighbors`, fit a KNN model on the training data and evaluate its performance on the test set using `accuracy_score`.

In [72]:
# Your code here...
# step 1. initialize the model
# using the best_value_for_k which was defined above with wine_tune_grid.best_params_ 
knn = KNeighborsClassifier(n_neighbors= best_value_for_k) 

# step 2. define our X and y
X = wine_train.iloc[:, :-1]
y = wine_train["class"]

#step 3. fit model to the train data
knn.fit(X,y)

# step 4. evaluate performance on the test set using accuracy_score
knn.score(wine_test.iloc[:, :-1], wine_test["class"])


0.9777777777777777

# Criteria


| **Criteria**                                           | **Complete**                                      | **Incomplete**                                    |
|--------------------------------------------------------|---------------------------------------------------|--------------------------------------------------|
| **Data Inspection**                                    | Data is inspected for number of variables, observations and data types. | Data inspection is missing or incomplete.         |
| **Data Scaling**                                       | Data scaling or normalization is applied where necessary (e.g., using `StandardScaler`). | Data scaling or normalization is missing or incorrectly applied. |
| **Model Initialization**                               | The KNN model is correctly initialized and a random seed is set for reproducibility.            | The KNN model is not initialized, is incorrect, or lacks a random seed for reproducibility. |
| **Parameter Grid for `n_neighbors`**                   | The parameter grid for `n_neighbors` is correctly defined. | The parameter grid is missing or incorrectly defined. |
| **Cross-Validation Setup**                             | Cross-validation is set up correctly with 10 folds. | Cross-validation is missing or incorrectly set up. |
| **Best Hyperparameter (`n_neighbors`) Selection**       | The best value for `n_neighbors` is identified using the grid search results. | The best `n_neighbors` is not selected or incorrect. |
| **Model Evaluation on Test Data**                      | The model is evaluated on the test data using accuracy. | The model evaluation is missing or uses the wrong metric. |


## Submission Information

🚨 **Please review our [Assignment Submission Guide](https://github.com/UofT-DSI/onboarding/blob/main/onboarding_documents/submissions.md)** 🚨 for detailed instructions on how to format, branch, and submit your work. Following these guidelines is crucial for your submissions to be evaluated correctly.

### Note:

If you like, you may collaborate with others in the cohort. If you choose to do so, please indicate with whom you have worked with in your pull request by tagging their GitHub username. Separate submissions are required.

### Submission Parameters:
* Submission Due Date: `11:59 PM - 11/24/2025`
* The branch name for your repo should be: `assignment-1`
* What to submit for this assignment:
    * This Jupyter Notebook (assignment_1.ipynb) should be populated and should be the only change in your pull request.
* What the pull request link should look like for this assignment: `https://github.com/<your_github_username>/LCR/pull/<pr_id>`
    * Open a private window in your browser. Copy and paste the link to your pull request into the address bar. Make sure you can see your pull request properly. This helps the technical facilitator and learning support staff review your submission easily.

Checklist:
- [ ] Created a branch with the correct naming convention.
- [ ] Ensured that the repository is public.
- [ ] Reviewed the PR description guidelines and adhered to them.
- [ ] Verify that the link is accessible in a private browser window.

If you encounter any difficulties or have questions, please don't hesitate to reach out to our team via our Slack at `#dc2-help`. Our Technical Facilitators and Learning Support staff are here to help you navigate any challenges.
